# Unified Bedrock Model Catalog Demonstration

This notebook demonstrates the **BedrockModelCatalog** - a comprehensive solution that provides unified access to Amazon Bedrock model data and CRIS (Cross-Region Inference Service) data in a single interface.

## Overview

The BedrockModelCatalog provides:
- 🎯 **Single Source of Truth**: Unified view of model availability across regions
- 🔄 **Automatic Integration**: Correlates regular model data with CRIS data
- 🌍 **Access Method Detection**: Identifies direct, regional_cris, or global_cris
- 📊 **Comprehensive Querying**: Rich filtering and analysis capabilities
- ⚡ **Smart Access Info**: Complete model access information with rationale

## What's New?

This notebook has been updated to use the new **BedrockModelCatalog** which replaces the deprecated `UnifiedModelManager`. Key improvements:
- ✅ API-only data retrieval (no HTML parsing)
- ✅ Automatic initialization (no manual refresh calls)
- ✅ Unified model and CRIS data in single structure
- ✅ Better error handling and fallback mechanisms
- ✅ Lambda-friendly design with configurable caching

## Setup and Imports

First, let's import the required modules and configure logging.

In [ ]:
# Import required modules
import sys
import logging
from pathlib import Path
from datetime import datetime
import json

# Add the src directory to the Python path
sys.path.append('../src')

# Import the new BedrockModelCatalog
from bestehorn_llmmanager.bedrock.catalog import BedrockModelCatalog, CacheMode
from bestehorn_llmmanager.bedrock.exceptions.llm_manager_exceptions import CatalogUnavailableError

# Configure logging to see what's happening
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("✅ BedrockModelCatalog imported successfully!")

## 1. Initialize the Unified Model Catalog

Let's initialize the BedrockModelCatalog with `force_refresh=True` to fetch fresh data.

**Note**: The catalog automatically handles:
- Fetching both model and CRIS data from AWS APIs
- Correlating and merging the data sources
- Caching for performance
- Fallback to bundled data if APIs are unavailable

In [ ]:
# Initialize the catalog with force refresh for demonstration
print("🔄 Initializing BedrockModelCatalog with fresh unified data...")

try:
    catalog = BedrockModelCatalog(
        force_refresh=True,  # Always fetch fresh data for demo
        timeout=60,          # Longer timeout for reliability
        fallback_to_bundled=True  # Fallback if API fails
    )
    
    print("✅ BedrockModelCatalog initialized successfully!")
    print(f"   Cache mode: {catalog.cache_mode.value}")
    print(f"   Catalog loaded: {catalog.is_catalog_loaded}")
    
except CatalogUnavailableError as e:
    print(f"❌ Failed to initialize catalog: {e}")
    print("💡 Check your AWS credentials and network connectivity")
    raise

## 2. Display Catalog Metadata

Let's examine the metadata to understand where the unified data came from.

In [ ]:
# Get catalog metadata
metadata = catalog.get_catalog_metadata()

print("📊 Unified Catalog Metadata:")
print("=" * 60)
print(f"   Source: {metadata.source.value}")
print(f"   Retrieved: {metadata.retrieval_timestamp}")
print(f"   Regions Queried: {len(metadata.api_regions_queried)}")
print(f"   Region List: {', '.join(metadata.api_regions_queried[:5])}...")

if metadata.cache_file_path:
    print(f"   Cache File: {metadata.cache_file_path}")
    
if metadata.bundled_data_version:
    print(f"   Bundled Version: {metadata.bundled_data_version}")

## 3. Basic Catalog Overview

Let's explore the basic information about our unified catalog.

In [ ]:
# Basic catalog information
print("📋 Unified Catalog Overview")
print("=" * 60)

# List all models
all_models = catalog.list_models()
print(f"\n🤖 Total Models: {len(all_models)}")

# Show first 15 models
print("\nSample Models:")
for i, model in enumerate(all_models[:15], 1):
    print(f"   {i:2d}. {model.model_name}")
if len(all_models) > 15:
    print(f"   ... and {len(all_models) - 15} more")

# Provider breakdown
print(f"\n🏢 Provider Analysis:")
providers = ["Amazon", "Anthropic", "Meta", "Mistral", "DeepSeek", "Writer"]
for provider in providers:
    provider_models = catalog.list_models(provider=provider)
    if provider_models:
        print(f"   {provider:12s}: {len(provider_models):2d} models")

## 4. Unified Model Access Information

This is the heart of the unified system - the `get_model_info()` method returns comprehensive access information including:
- Direct model ID (if available)
- Inference profile ID (if available via CRIS)
- **Access method** (direct, regional_cris, global_cris)
- Streaming support
- Input/output modalities

The access method field tells you how to access the model in that region.

In [ ]:
def display_unified_access_info(model_name: str, region: str):
    """Display comprehensive unified access information for a model-region pair."""
    print(f"\n🔍 Unified Access Info: '{model_name}' in '{region}'")
    print("-" * 60)
    
    # Check availability
    available = catalog.is_model_available(model_name=model_name, region=region)
    print(f"   Available: {'✅ Yes' if available else '❌ No'}")
    
    if not available:
        return
    
    # Get unified access information
    access_info = catalog.get_model_info(model_name=model_name, region=region)
    if access_info:
        print(f"   Access Method: {access_info.access_method.value.upper()}")
        
        if access_info.model_id:
            print(f"   🎯 Model ID: {access_info.model_id}")
        
        if access_info.inference_profile_id:
            print(f"   🔄 Inference Profile: {access_info.inference_profile_id}")
        
        print(f"   Streaming: {'✅' if access_info.supports_streaming else '❌'}")
        print(f"   Input Modalities: {', '.join(access_info.input_modalities)}")
        print(f"   Output Modalities: {', '.join(access_info.output_modalities)}")

# Test cases demonstrating different access patterns
test_cases = [
    ("Claude 3 Haiku", "us-east-1"),      # Should have both direct and CRIS
    ("Nova Lite", "us-east-1"),          # Amazon model
    ("Nova Pro", "eu-west-1"),           # Cross-region availability
    ("Claude 3.5 Sonnet", "us-west-2"),  # Popular model
    ("Claude 3 Haiku", "ap-southeast-1"), # CRIS access in APAC
]

print("🎯 Unified Model Access Information Examples")
print("=" * 60)

for model_name, region in test_cases:
    display_unified_access_info(model_name, region)

## 5. Understanding Access Methods

The unified catalog provides an **access_method** field that tells you how to access each model:

- **direct**: Use the model ID directly in the region (e.g., `anthropic.claude-3-haiku...`)
- **regional_cris**: Use a regional inference profile (e.g., `us.anthropic.claude-3-haiku`)
- **global_cris**: Use a global inference profile (e.g., `anthropic.claude-3-haiku`)

Let's examine the access methods for different models.

In [ ]:
# Examine access methods
print("🔑 Model Access Methods Analysis:")
print("=" * 60)

example_models = ["Claude 3 Haiku", "Claude 3.5 Sonnet", "Nova Lite", "Nova Pro"]
example_region = "us-east-1"

for model_name in example_models:
    model_info = catalog.get_model_info(model_name=model_name, region=example_region)
    
    if model_info:
        print(f"\n📋 {model_name}:")
        print(f"   Model ID: {model_info.model_id}")
        print(f"   Access Method: {model_info.access_method.value}")
        
        if model_info.inference_profile_id:
            print(f"   Inference Profile: {model_info.inference_profile_id}")
            print(f"   💡 Can use inference profile for cross-region routing")
        else:
            print(f"   💡 Use model ID for direct access in region")

## 6. Regional Analysis with Unified Data

Let's analyze model availability across different regions using the unified data structure.

In [ ]:
# Regional analysis
print("🌍 Regional Availability Analysis:")
print("=" * 60)

regions_to_analyze = ["us-east-1", "us-west-2", "eu-west-1", "ap-southeast-1"]

for region in regions_to_analyze:
    region_models = catalog.list_models(region=region)
    print(f"\n📍 {region}: {len(region_models)} models")
    
    # Count by access method
    access_methods = {}
    for model in region_models:
        model_info = catalog.get_model_info(model_name=model.model_name, region=region)
        if model_info:
            method = model_info.access_method.value
            access_methods[method] = access_methods.get(method, 0) + 1
    
    print("   Access Methods:")
    for method, count in sorted(access_methods.items()):
        print(f"      {method}: {count} models")

## 7. Filter Streaming-Capable Models

Let's find all models that support streaming responses.

In [ ]:
# Filter streaming-capable models
print("🌊 Streaming-Capable Models:")
print("=" * 60)

streaming_models = catalog.list_models(streaming_only=True)
print(f"\nTotal: {len(streaming_models)} streaming-capable models")

# Show first 10
print("\nExamples:")
for i, model in enumerate(streaming_models[:10], 1):
    print(f"   {i}. {model.model_name} ({model.provider})")
    
if len(streaming_models) > 10:
    print(f"   ... and {len(streaming_models) - 10} more")

## 8. Combine Multiple Filters

Let's combine multiple filters to find specific models.

In [ ]:
# Combine filters: Anthropic models in us-west-2 that support streaming
print("🔍 Combined Filter Example:")
print("   Provider: Anthropic")
print("   Region: us-west-2")
print("   Streaming: Required")
print("=" * 60)

filtered_models = catalog.list_models(
    provider="Anthropic",
    region="us-west-2",
    streaming_only=True
)

print(f"\nFound: {len(filtered_models)} models")

for model in filtered_models:
    # Get access info to show access method
    access_info = catalog.get_model_info(model_name=model.model_name, region="us-west-2")
    if access_info:
        print(f"   • {model.model_name} (access: {access_info.access_method.value})")

## 9. Troubleshooting

Common issues and solutions when working with BedrockModelCatalog.

### Import Errors

If you encounter import errors:

```python
# ❌ ImportError: No module named 'bestehorn_llmmanager'
```

**Solutions**:
1. Ensure you're running from the `notebooks/` directory
2. Check that `sys.path.append('../src')` is executed
3. Verify the package is installed: `pip install -e .` from project root

### API Timeout Errors

If API fetching times out:

```python
# Increase timeout and enable fallback
catalog = BedrockModelCatalog(
    force_refresh=True,
    timeout=120,  # Longer timeout
    fallback_to_bundled=True  # Use bundled data if API fails
)
```

### Cache Permission Errors

If you can't write to the cache directory:

```python
# Use memory-only caching
catalog = BedrockModelCatalog(
    cache_mode=CacheMode.MEMORY,
    force_refresh=True
)
```

### Model Not Found

If a model isn't found:

```python
# Check available models
all_models = catalog.list_models()
model_names = [m.model_name for m in all_models]
print("Available models:", model_names)

# Try different name variations
# The catalog supports flexible name matching
variations = [
    "Claude 3 Haiku",
    "claude-3-haiku",
    "anthropic.claude-3-haiku-20240307-v1:0"
]
```

## 10. Summary and Best Practices

Summary of BedrockModelCatalog unified capabilities and usage recommendations.

In [ ]:
print("📋 BedrockModelCatalog Unified Summary:")
print("=" * 60)

print("\n✅ Key Capabilities Demonstrated:")
features = [
    "Unified API-based model and CRIS data access",
    "Automatic initialization (no manual refresh calls)",
    "Comprehensive model information with access methods",
    "Access method detection (direct, regional_cris, global_cris)",
    "Flexible filtering by provider, region, and streaming support",
    "Model availability checking across regions",
    "Regional analysis with unified data structure",
    "Catalog metadata for source and freshness tracking",
    "Configurable caching strategies (FILE, MEMORY, NONE)",
    "Bundled fallback data for offline scenarios"
]

for i, feature in enumerate(features, 1):
    print(f"   {i}. {feature}")

print("\n💡 Best Practices:")
practices = [
    "Use force_refresh=True for demos to ensure fresh data",
    "Enable fallback_to_bundled=True for reliability",
    "Handle CatalogUnavailableError exceptions properly",
    "Check access_method field to determine optimal access pattern",
    "Use inference profiles for cross-region routing",
    "Leverage unified data structure for regional analysis",
    "Monitor catalog metadata to understand data source",
    "Use appropriate cache modes for your use case"
]

for i, practice in enumerate(practices, 1):
    print(f"   {i}. {practice}")

print("\n🎯 Use Cases:")
use_cases = [
    "Unified model availability checking across regions",
    "Cross-region inference setup with CRIS profiles",
    "Access method optimization (direct vs CRIS)",
    "Multi-region deployment planning",
    "Cost optimization through region selection",
    "Disaster recovery and failover configuration",
    "Comprehensive model catalog management"
]

for i, use_case in enumerate(use_cases, 1):
    print(f"   {i}. {use_case}")

print("\n📚 Additional Resources:")
resources = [
    "Migration Guide: docs/MIGRATION_GUIDE.md",
    "API Reference: docs/forLLMConsumption.md",
    "Code Examples: examples/catalog_*.py",
    "README: README.md (BedrockModelCatalog section)"
]

for resource in resources:
    print(f"   • {resource}")

print("\n🎉 BedrockModelCatalog Unified Demo Complete!")
print("   Ready for production use with comprehensive unified model management.")